# Black Scholes Exercise 3: Numexpr implementation

- Use numexpr
- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code

In [ ]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

# The numexpr Black Scholes algorithm

How does this differ from the NumPy and Naive variants?

In [ ]:
import numexpr as ne

def black_scholes (nopt, price, strike, t, rate, vol ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = ne.evaluate("log(P / S) ")
    b = ne.evaluate("T * mr ")

    z = ne.evaluate("T * sig_sig_two ")
    c = ne.evaluate("0.25 * z ")
    y = ne.evaluate("1/sqrt(z) ")

    w1 = ne.evaluate("(a - b + c) * y ")
    w2 = ne.evaluate("(a - b - c) * y ")

    d1 = ne.evaluate("0.5 + 0.5 * erf(w1) ")
    d2 = ne.evaluate("0.5 + 0.5 * erf(w2) ")

    Se = ne.evaluate("exp(b) * S ")

    call = ne.evaluate("P * d1 - Se * d2 ")
    # TODO convert put = call - P + Se to numexpr

    return call, put

ne.set_num_threads(ne.detect_number_of_cores())
ne.set_vml_accuracy_mode('high')

## Run timeit, cProfile, and/or VTune to see what is happening

In [ ]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

## Crunching the commands down

What is different in this variant of Black Scholes?

In [ ]:
def black_scholes(price, strike, t, rate, vol ):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    P = price
    S = strike
    T = t

    call = ne.evaluate("P * (0.5 + 0.5 * erf((log(P / S) - T * mr + 0.25 * T * sig_sig_two) * 1/sqrt(T * sig_sig_two))) - S * exp(T * mr) * (0.5 + 0.5 * erf((log(P / S) - T * mr - 0.25 * T * sig_sig_two) * 1/sqrt(T * sig_sig_two))) ")
    put = ne.evaluate("call - P + S * exp(T * mr) ")
    # TODO compute put using numexpr
    #      Note: there is no variable "Se"
    
    return call, put
        

ne.set_num_threads(ne.detect_number_of_cores())
ne.set_vml_accuracy_mode('high')

## Run timeit, cProfile, and/or VTune to see what is happening